In [87]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from google.colab                import drive

pd.set_option('display.max_columns', 500)

In [88]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [89]:
df_am = pd.read_csv('/content/gdrive/MyDrive/Datasets/base_covid_AM/base.csv', sep=';', low_memory=False, parse_dates=['_dt_notificacao','_dt_evolucao','_dt_sintomas'])
df_am = df_am.drop(['_etnia', '_faixa etária', '_raca',
                    '_conclusao', '_se_notificacao', '_evolução','_origem',
                    '_distrito', '_bairro_mapa','_dt_evolucao', '_dt_notificacao', '_dt_sintomas'], axis = 1)

## Tratamento de dados

In [90]:
df_am[['_comorb_cardio', '_comorb_cromossomica',
       '_comorb_diabetes', '_comorb_hemato', '_comorb_hepatica',
       '_comorb_imuno', '_comorb_neurologica', '_comorb_obessidade',
       '_comorb_renal', '_comorb_respiratoria','_gestante', '_profiss_saude',
       '_sintoma_dispneia', '_sintoma_febre', '_sintoma_garganta',
       '_sintoma_tosse','_sintoma_olfato', '_sintoma_paladar','_sintoma_outros']] = df_am[['_comorb_cardio', '_comorb_cromossomica',
       '_comorb_diabetes', '_comorb_hemato', '_comorb_hepatica',
       '_comorb_imuno', '_comorb_neurologica', '_comorb_obessidade',
       '_comorb_renal', '_comorb_respiratoria','_gestante', '_profiss_saude',
       '_sintoma_dispneia', '_sintoma_febre', '_sintoma_garganta',
       '_sintoma_tosse','_sintoma_olfato', '_sintoma_paladar', '_sintoma_outros']].fillna('Não')

In [91]:
df_am = df_am[(df_am['_idade'] >= 18.0) & (df_am['_idade'] <= 100.0)]
df_am['_idade'] = df_am['_idade'].fillna(df_am['_idade'].mean())
#df_am = df_am.dropna()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Feature Engineering

In [92]:
df_am['grupo_de_risco'] = df_am['_idade'] >= 60.0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
df_am.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 571056 entries, 1 to 634837
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   _classificacao        571056 non-null  object 
 1   _comorb_cardio        571056 non-null  object 
 2   _comorb_cromossomica  571056 non-null  object 
 3   _comorb_diabetes      571056 non-null  object 
 4   _comorb_hemato        571056 non-null  object 
 5   _comorb_hepatica      571056 non-null  object 
 6   _comorb_imuno         571056 non-null  object 
 7   _comorb_neurologica   571056 non-null  object 
 8   _comorb_obessidade    571056 non-null  object 
 9   _comorb_renal         571056 non-null  object 
 10  _comorb_respiratoria  571056 non-null  object 
 11  _criterio             122779 non-null  object 
 12  _idade                571056 non-null  float64
 13  _gestante             571056 non-null  object 
 14  _profiss_saude        571056 non-null  object 
 15  

In [94]:
df_am['_srag'].value_counts()

SRAG por covid-19                    22753
SRAG não especificado                 6337
SRAG por outro vírus respiratório      113
SRAG por outro agente etiológico        29
SRAG por influenza                      23
Name: _srag, dtype: int64

In [95]:
df_am['_teste_antigeno'].value_counts()

1.0      91167
2.0       5581
3.0        633
4.0        114
5.0         35
6.0         17
7.0         10
8.0          7
10.0         6
9.0          4
15.0         1
13.0         1
12.0         1
11.0         1
115.0        1
21.0         1
17.0         1
Name: _teste_antigeno, dtype: int64

In [96]:
df_am.head(20)

,_classificacao,_comorb_cardio,_comorb_cromossomica,_comorb_diabetes,_comorb_hemato,_comorb_hepatica,_comorb_imuno,_comorb_neurologica,_comorb_obessidade,_comorb_renal,_comorb_respiratoria,_criterio,_idade,_gestante,_profiss_saude,_sexo,_sintoma_dispneia,_sintoma_febre,_sintoma_garganta,_sintoma_tosse,_bairro,_sintoma_olfato,_sintoma_paladar,_sintoma_outros,_srag,_taxa,_teste_anticorpo,_teste_antigeno,_teste_pcr,_srag_agente,grupo_de_risco
1,Confirmado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Laboratorial,26.0,Não,Não,F,Sim,Sim,Sim,Sim,TARUMA-ACU,Não,Não,Sim,SRAG por influenza,"0,684931507",NaN,1.0,1.0,Influenza B,False
2,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Laboratorial,27.0,Não,Não,M,Sim,Sim,Sim,Sim,JORGE TEIXEIRA,Não,Não,Sim,SRAG não especificado,"0,073137374",NaN,1.0,1.0,NaN,False
3,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Laboratorial,94.0,Não,Não,M,Sim,Sim,Sim,Sim,FLORES,Não,Não,Sim,SRAG não especificado,"0,198306463",NaN,1.0,1.0,NaN,True
8,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Laboratorial,28.0,Não,Não,F,Sim,Sim,Não,Sim,CIDADE DE DEUS,Não,Não,Sim,SRAG não especificado,"0,117699678",NaN,1.0,1.0,NaN,False
9,Descartado,Sim,Não,Não,Não,Não,Não,Não,Não,Não,Não,Laboratorial,80.0,Não,Não,M,Sim,Sim,Não,Sim,PETROPOLIS,Não,Não,Sim,SRAG não especificado,"0,232948192",NaN,1.0,1.0,NaN,True
10,Confirmado,Sim,Não,Sim,Não,Não,Não,Não,Não,Não,Sim,Laboratorial,74.0,Não,Não,M,Sim,Sim,Sim,Sim,CACHOEIRINHA,Não,Não,Sim,SRAG por influenza,"0,514403292",NaN,1.0,1.0,Influenza B,True
11,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Critério Clínico,23.0,Sim,Não,F,Não,Sim,Sim,Sim,SANTA ETELVINA,Não,Não,Sim,SRAG não especificado,"0,314386318",NaN,NaN,NaN,NaN,False
12,Confirmado,Não,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Laboratorial,26.0,Não,Não,F,Sim,Sim,Sim,Sim,MONTE DAS OLIVEIRAS,Não,Não,Sim,SRAG por influenza,"0,205558296",1.0,1.0,1.0,Influenza B,False
14,Descartado,Não,Não,Sim,Não,Não,Não,Não,Não,Não,Não,Laboratorial,85.0,Não,Não,F,Sim,Sim,Não,Sim,PONTA NEGRA,Não,Não,Sim,SRAG não especificado,"1,648804617",NaN,1.0,1.0,NaN,True
18,Descartado,Não,Não,Não,Não,Não,Não,Não,Não,Não,Não,Laboratorial,49.0,Não,Não,M,Sim,Sim,Sim,Sim,TARUMA,Não,Não,Sim,SRAG não especificado,"0,243042897",NaN,1.0,1.0,NaN,False


In [97]:
path = '/content/gdrive/MyDrive/Datasets/base_covid_AM'
#df_am.to_csv(path + '/BASE_COVID_AM.csv', index=False)